In [1]:
import pandas as pd
import numpy as np
import json
import warnings
import spacy
warnings.filterwarnings("ignore")
import random
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.tokens import DocBin

In [2]:
train_data = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\train.csv")
features = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\features.csv")
patient_notes = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\patient_notes.csv")

In [3]:
new_train = train_data[(train_data.feature_num == 9)][['pn_num','annotation',"location","feature_num"]]
new_train = new_train[new_train.annotation != '[]']

new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

new_train = (new_train.set_index(['pn_num',"feature_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))

new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])

row_index = []
start_location = []
end_location = []
pn_notes = set()
drop_index = []
for index,row in new_train.iterrows():   # Iterating every row in trainn data
    if row["pn_num"] not in pn_notes:    # Verifying whether there is a change in pn_num to reinitialize start and end location lists
        start_location = []
        end_location = []
        pn_notes.add(row["pn_num"])      # Appending pn_num
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
    else: 
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
            if row["start_location"] in start_location:
                if row["end_location"] >= new_train.iloc[index -1,5]:
                    drop_index.append(index -1)
                else:
                    drop_index.append(index)
            elif row["end_location"] in end_location:
                if row["start_location"] <= new_train.iloc[index -1,4]:
                    drop_index.append(index - 1)
                else:
                    drop_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
        
new_train = new_train.drop(drop_index,axis=0)

pn_Num_list = list(new_train.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in new_train[new_train.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_"+ str(row["feature_num"]))) # store the locations in numeric format
        
    text = patient_notes[patient_notes.pn_num == patient_number]["pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_feature9.json", "w") as outfile:
    outfile.write(json_object)
    
import json
 
# Opening JSON file
f = open("sample_feature9.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

train = data[:int(len(data)*0.8)]
validation = data[int(len(data)*0.8):]

# converting the json file to spacy file in order to use it in training model

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

#train data set

camp_train = create_training(train)
camp_train.to_disk("nbme_train_feature9.spacy")

camp_train = create_training(validation)
camp_train.to_disk("nbme_validation_feature9.spacy")

!python -m spacy init fill-config ./base_config_feature9.cfg ./config_feature9.cfg

!python -m spacy train config_feature9.cfg --output ./output9

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 770.27it/s]


[+] Auto-filled config with all values
[+] Saved config
config_feature9.cfg
You can now add your data and train your pipeline:
python -m spacy train config_feature9.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-03-31 12:58:16.867099: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-31 12:58:16.867358: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[i] Saving to output directory: output9
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     48.67    0.00    0.00    0.00    0.00
  2     200      10077.00   2710.60   56.60   60.00   53.57    0.57
  5     400         83.95    159.30   68.97   66.67   71.43    0.69
  7     600        122.35     95.41   75.00   75.00   75.00    0.75
 10     800         42.65     42.89   77.78   80.77   75.00    0.78
 12    1000        143.98     35.41   77.78   80.77   75.00    0.78
 15    1200         35.32     20.19   69.09   70.37   67.86    0.69
 17    1400         41.87     22.39   74.07   76.92   71.43    0.74
 20    1600         

2022-03-31 12:58:21.003960: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-31 12:58:21.004015: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-03-31 12:58:23,545] [INFO] Set up nlp object from config
[2022-03-31 12:58:23,554] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-31 12:58:23,558] [INFO] Created vocabulary
[2022-03-31 12:58:23,559] [INFO] Finished initializing nlp object
[2022-03-31 12:58:24,094] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [4]:
new_train = train_data[(train_data.feature_num == 12)][['pn_num','annotation',"location","feature_num"]]
new_train = new_train[new_train.annotation != '[]']

new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

new_train = (new_train.set_index(['pn_num',"feature_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))

new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])

row_index = []
start_location = []
end_location = []
pn_notes = set()
drop_index = []
for index,row in new_train.iterrows():   # Iterating every row in trainn data
    if row["pn_num"] not in pn_notes:    # Verifying whether there is a change in pn_num to reinitialize start and end location lists
        start_location = []
        end_location = []
        pn_notes.add(row["pn_num"])      # Appending pn_num
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
    else: 
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
            if row["start_location"] in start_location:
                if row["end_location"] >= new_train.iloc[index -1,5]:
                    drop_index.append(index -1)
                else:
                    drop_index.append(index)
            elif row["end_location"] in end_location:
                if row["start_location"] <= new_train.iloc[index -1,4]:
                    drop_index.append(index - 1)
                else:
                    drop_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
        
new_train = new_train.drop(drop_index,axis=0)

pn_Num_list = list(new_train.pn_num.unique())

final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in new_train[new_train.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_"+ str(row["feature_num"]))) # store the locations in numeric format
        
    text = patient_notes[patient_notes.pn_num == patient_number]["pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list
    
    
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_feature12.json", "w") as outfile:
    outfile.write(json_object)
    
import json
 
# Opening JSON file
f = open("sample_feature12.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

train = data[:int(len(data)*0.8)]
validation = data[int(len(data)*0.8):]

# converting the json file to spacy file in order to use it in training model

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

#train data set

camp_train = create_training(train)
camp_train.to_disk("nbme_train_feature12.spacy")

camp_train = create_training(validation)
camp_train.to_disk("nbme_validation_feature12.spacy")

!python -m spacy init fill-config ./base_config_feature12.cfg ./config_feature12.cfg

!python -m spacy train config_feature12.cfg --output ./output12

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 653.70it/s]

Skipping entity
Skipping entity
Skipping entity


[+] Auto-filled config with all values
[+] Saved config
config_feature12.cfg
You can now add your data and train your pipeline:
python -m spacy train config_feature12.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-03-31 13:00:52.648006: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-31 13:00:52.648048: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[i] Saving to output directory: output12
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     74.33    0.00    0.00    0.00    0.00
  2     200          4.82   1289.95   97.30  100.00   94.74    0.97
  5     400          0.00      0.00   94.44  100.00   89.47    0.94
  8     600          0.00      0.00   94.44  100.00   89.47    0.94
 10     800          0.00      0.00   94.44  100.00   89.47    0.94
 13    1000          0.00      0.00   94.44  100.00   89.47    0.94
 16    1200          0.00      0.00   94.44  100.00   89.47    0.94
 18    1400          0.00      0.00   94.44  100.00   89.47    0.94
 21    1600        

2022-03-31 13:00:56.692185: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-31 13:00:56.692296: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-03-31 13:00:59,271] [INFO] Set up nlp object from config
[2022-03-31 13:00:59,278] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-31 13:00:59,281] [INFO] Created vocabulary
[2022-03-31 13:00:59,282] [INFO] Finished initializing nlp object
[2022-03-31 13:00:59,713] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
